In [ ]:
# Import all classes of PuLP module
from pulp import *
import pandas as pd

In [ ]:
# Initialize Class set minimize as objective
model = LpProblem("PoliceSchedulingProb", LpMinimize)

# Create 28 Shifts (4 shifts per day, 7 days a week)
shifts = list(range(28))

In [ ]:
# Define Decision Variables
x = LpVariable.dicts('officer_shift', shifts, lowBound=0, cat='Integer')
y = LpVariable.dicts('detective_shift', shifts, lowBound=0, cat='Integer')

In [ ]:
# Define Objective: Total number of police on shift
model += lpSum([x[i] for i in shifts]) + lpSum([y[i] for i in shifts])

In [ ]:
"""
Constraint 1: No. officers check-in everyday on weekdays
"""
model += x[0]+x[1]+x[2]+x[3]>=20 #Monday
model += x[4]+x[5]+x[6]+x[7]>=18 #Tuesday
model += x[8]+x[9]+x[10]+x[11]>=15 #Wednesday
model += x[12]+x[13]+x[14]+x[15]>=18 #Thursday
model += x[16]+x[17]+x[18]+x[19]>=25 #Friday

"""
Constraint 2: No. officers check-in everyday on weekends
"""
model += x[20]+x[21]+x[22]+x[23]>=10 #Saturday
model += x[24]+x[25]+x[26]+x[27]>=9 #Sunday

In [ ]:
"""
Constraint 3: No. detectives check-in everyday on weekdays
"""
model += y[0]+y[1]+y[2]+y[3]>=10 #Monday
model += y[4]+y[5]+y[6]+y[7]>=9 #Tuesday
model += y[8]+y[9]+y[10]+y[11]>=8 #Wednesday
model += y[12]+y[13]+y[14]+y[15]>=9 #Thursday
model += y[16]+y[17]+y[18]+y[19]>=12 #Friday

"""
Constraint 4: No. detectives check-in everyday on weekends
"""
model += y[20]+y[21]+y[22]+y[23]>=5 #Saturday
model += y[24]+y[25]+y[26]+y[27]>=4 #Sunday

In [ ]:
"""
Constraint 5: No. of officers and detectives standby on each shift
"""

# Shift numbers belong to each daytime shift
midnight_shift = [0,4,8,12,16,20,24]
morning_shift = [1,5,9,13,17,21,25]
afternoon_shift = [2,6,10,14,18,22,26]
night_shift = [3,7,11,15,19,23,27]

for i in midnight_shift:
	#At least 2 officers standby on midnight_shift
	model += x[i]>=1
	#At least 1 detective standby on midnight_shift
	model += y[i]>=1

for i in morning_shift:
	#At least 4 officers standby on morning_shift
	model += x[i]>=4
	#At least 3 detective2 standby on morning_shift
	model += y[i]>=3

for i in afternoon_shift:
	#At least 5 officers standby on afternoon_shift
	model += x[i]>=5
	#At least 4 detective2 standby on afternoon_shift
	model += y[i]>=4

for i in night_shift:
	#At least 3 officers standby on night_shift
	model += x[i]>=3
	#At least 2 detective2 standby on night_shift
	model += y[i]>=2

In [ ]:
# The problem is solved using PuLP's choice of Solver
model.solve()

shift_list = []
assign_list = []

# Print the variables optimized value
for v in model.variables():
	shift_list.append(v.name)
	assign_list.append(v.varValue)
	print(f"{v.varValue}, {v.name}")


1.0, detective_shift_0
3.0, detective_shift_1
4.0, detective_shift_10
2.0, detective_shift_11
1.0, detective_shift_12
3.0, detective_shift_13
4.0, detective_shift_14
2.0, detective_shift_15
3.0, detective_shift_16
3.0, detective_shift_17
4.0, detective_shift_18
2.0, detective_shift_19
4.0, detective_shift_2
1.0, detective_shift_20
3.0, detective_shift_21
4.0, detective_shift_22
2.0, detective_shift_23
1.0, detective_shift_24
3.0, detective_shift_25
4.0, detective_shift_26
2.0, detective_shift_27
2.0, detective_shift_3
1.0, detective_shift_4
3.0, detective_shift_5
4.0, detective_shift_6
2.0, detective_shift_7
1.0, detective_shift_8
3.0, detective_shift_9
8.0, officer_shift_0
4.0, officer_shift_1
7.0, officer_shift_10
3.0, officer_shift_11
6.0, officer_shift_12
4.0, officer_shift_13
5.0, officer_shift_14
3.0, officer_shift_15
13.0, officer_shift_16
4.0, officer_shift_17
5.0, officer_shift_18
3.0, officer_shift_19
5.0, officer_shift_2
1.0, officer_shift_20
4.0, officer_shift_21
5.0, offic

In [ ]:
schedule_result = pd.DataFrame({'shift_name': shift_list, 'assign_no': assign_list})
schedule_result[['police_type', 'shift','shift_no']] = schedule_result['shift_name'].str.split('_', expand=True)
schedule_result = (schedule_result
				   .drop(['shift_name', 'shift'],axis=1)
				   .astype({'assign_no': 'int', 'shift_no':'int'})
				   )
schedule_result.head()

,assign_no,police_type,shift_no
0,1,detective,0
1,3,detective,1
2,4,detective,10
3,2,detective,11
4,1,detective,12


In [ ]:
midnight_shift = [0,4,8,12,16,20,24]
morning_shift = [1,5,9,13,17,21,25]
afternoon_shift = [2,6,10,14,18,22,26]
night_shift = [3,7,11,15,19,23,27]

schedule_result['shift_number'] = schedule_result['shift_no']
schedule_result['shift_no'] = (schedule_result['shift_no']
							   .replace(midnight_shift, '00:00-06:00')
							   .replace(morning_shift, '06:00-12:00')
							   .replace(afternoon_shift, '12:00-18:00')
							   .replace(night_shift, '18:00-00:00')
							   )


schedule_result['shift_number'] = (schedule_result['shift_number']
							   .replace(list(range(0, 4, 1)), 'monday')
							   .replace(list(range(4, 8, 1)), 'tuesday')
							   .replace(list(range(8, 12, 1)), 'wednesday')
							   .replace(list(range(12, 16, 1)), 'thursday')
							   .replace(list(range(16, 20, 1)), 'friday')
							   .replace(list(range(20, 24, 1)), 'saturday')
							   .replace(list(range(24, 28, 1)), 'sunday')
							   )

schedule_result.head()

,assign_no,police_type,shift_no,shift_number
0,1,detective,00:00-06:00,monday
1,3,detective,06:00-12:00,monday
2,4,detective,12:00-18:00,wednesday
3,2,detective,18:00-00:00,wednesday
4,1,detective,00:00-06:00,thursday


In [ ]:
summary = schedule_result.pivot_table(values='assign_no',columns=['shift_number','police_type'], index='shift_no')
summary.head()

shift_number    friday            monday          saturday            sunday  \
police_type  detective officer detective officer detective officer detective   
shift_no                                                                       
00:00-06:00          3      13         1       8         1       1         1   
06:00-12:00          3       4         3       4         3       4         3   
12:00-18:00          4       5         4       5         4       5         4   
18:00-00:00          2       3         2       3         2       3         2   

shift_number          thursday           tuesday         wednesday          
police_type  officer detective officer detective officer detective officer  
shift_no                                                                    
00:00-06:00        1         1       6         1       6         1       1  
06:00-12:00        4         3       4         3       4         3       4  
12:00-18:00        5         4       5         4       5         4       7  
18:00-00:00        3         2       3         2       3         2       3

In [ ]:
summary.to_csv(r".\datasets\result.csv")